# Classifying Traffic Signs with OpenCV

For this project we will be trying split the traffic sign dataset into 3 separate classes:
- Stop signs
- Red Circles
- Blue rectangles/squares 

### Imports

In [57]:
import cv2 as cv
import numpy as np
import os, os.path
import shutil

### Useful functions

In [58]:
def get_file_list():
    file_list = []
    for root, _, files in os.walk('./dataset'):
	    for file in files:
		    file_list.append(os.path.join(root,file))
    return file_list

In [59]:
def process_image(segmented_img, original_img, color):
    segmented_img = smooth(segmented_img)
    if color == "blue":
        kernel = np.ones((5,5), np.uint8)
        segmented_img = cv.dilate(segmented_img, kernel, iterations=3)
        segmented_img = cv.erode(segmented_img, kernel, iterations=3)
        segmented_img = cv.Canny(segmented_img, 150, 200) 
        
    final, prediction = search_contours(segmented_img, original_img, color)
    
    return final, prediction

In [60]:
# Remove image noise with gaussian blur (better at preserving edges)
def smooth(img):
    return cv.GaussianBlur(img, (3,3), 0)


In [61]:
# Increases image brightness
def increase_brightness(img, value=30):
    hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    h, s, v = cv.split(hsv)

    lim = 255 - value
    v[v > lim] = 255
    v[v <= lim] += value

    final_hsv = cv.merge((h, s, v))
    img = cv.cvtColor(final_hsv, cv.COLOR_HSV2BGR)
    return img

In [62]:
# Add contrast to image by histogram equalization
def add_contrast(img):
    return cv.equalizeHist(img)

In [63]:
#  Segments reds and blues of an image
def segment(img, color):
    img_hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)
    if color == 'red':
        # lower mask (0-10)
        lower_red = np.array([0,90,50])
        upper_red = np.array([10,255,255])
        mask0 = cv.inRange(img_hsv, lower_red, upper_red)  

        # upper mask (160-180)
        lower_red = np.array([160,90,50])
        upper_red = np.array([180,255,255])
        mask1 = cv.inRange(img_hsv, lower_red, upper_red)

        mask = mask0 | mask1
    elif color == 'blue':
        lower_blue = np.array([100,160,50])
        upper_blue = np.array([130,255,255])
        mask = cv.inRange(img_hsv, lower_blue, upper_blue) 

    segmented_img = cv.bitwise_and(img_hsv, img_hsv, mask = mask)
    segmented_img = cv.cvtColor(segmented_img, cv.COLOR_BGR2GRAY)
    return segmented_img    


In [64]:
# Find the contour with the largest area in a gray scale image
def find_largest_contour(segmented_img, original_img):
    contours, _ = cv.findContours(segmented_img,2,1)
    cnt = contours
    big_contour = []
    max = 0
    for i in cnt:
        area = cv.contourArea(i)
        if(area > max):
            max = area
            big_contour = i 
    if max > 1000 and len(big_contour) > 25 and len(big_contour) < 1200:
        final = cv.drawContours(original_img, big_contour, -1, (0,255,0), 3)
    else:
        final = original_img

    return final, big_contour

In [65]:
## TODO: Improve sensibility to blue squares 
# Find a blue square
def search_contours(segmented_img, original_img, color):
    if cv.countNonZero(segmented_img) == 0:
        return segmented_img, 'unrecognised'
    final, contour = find_largest_contour(segmented_img, original_img)
    if len(contour) > 100:
        peri = cv.arcLength(contour, True)
        approx = cv.approxPolyDP(contour, 0.01 * peri, True)
        return final, classify_sign(approx, color)

    return final, 'unrecognised'

In [66]:
def classify_sign(approx_cnt, color):
    if color == 'blue':
        if len(approx_cnt) == 4:
            return 'blue_square'
    elif color == 'red':
        if len(approx_cnt) == 8:
            return 'stop_sign'
        elif len(approx_cnt > 8):
            return 'red_circle'
    return 'unrecognised'

## Red Circles

This is the section to detect red circles

After processing the image to gray values holding as the white value the red features a contour finder is used to detect an ellipse correctly. It can also be used in the case of stop signs to obtain octagons.

In [67]:
prediction = {}
shutil.rmtree('results/blue_squares')
shutil.rmtree('results/red_circles')
shutil.rmtree('results/stop_signs')
shutil.rmtree('results/unrecogniseds')
os.mkdir('results/blue_squares')
os.mkdir('results/red_circles')
os.mkdir('results/stop_signs')
os.mkdir('results/unrecogniseds')
for file in get_file_list():
    img = cv.imread(file)
    original = img.copy()
    img = smooth(img)
    img = increase_brightness(img)
    # cv.imshow("original", img)
    img_blue, img_red = segment(img, 'blue'), segment(img, 'red')
    
    # cv.imshow("filtered blue", img_blue)
    # cv.imshow("filtered red", img_red)
    
    final, prediction[file] = process_image(img_blue, img.copy(), 'blue')
    if prediction[file] == 'unrecognised':
        final, prediction[file] = process_image(img_red, img.copy(), 'red')

    if prediction[file] == 'unrecognised':
        final = original
        
    cv.imwrite('./results/' + prediction[file]+'s'+'/'+ file.split('/')[-1], final)
    # cv.imshow(prediction[file], final)
    # cv.waitKey(0)
    # cv.destroyAllWindows()
    
print(prediction)
cv.destroyAllWindows()
    

{'./dataset/road161.png': 'blue_square', './dataset/road143.png': 'blue_square', './dataset/road246.png': 'unrecognised', './dataset/road302.png': 'unrecognised', './dataset/road132.png': 'blue_square', './dataset/road204.png': 'blue_square', './dataset/road330.png': 'unrecognised', './dataset/road215.png': 'unrecognised', './dataset/road235.png': 'unrecognised', './dataset/road118.png': 'red_circle', './dataset/road76.png': 'stop_sign', './dataset/road182.png': 'unrecognised', './dataset/road344.png': 'stop_sign', './dataset/road321.png': 'unrecognised', './dataset/road221.png': 'unrecognised', './dataset/road133.png': 'blue_square', './dataset/road157.png': 'blue_square', './dataset/road115.png': 'red_circle', './dataset/road268.png': 'unrecognised', './dataset/road212.png': 'unrecognised', './dataset/road89.png': 'red_circle', './dataset/road91.png': 'stop_sign', './dataset/road96.png': 'stop_sign', './dataset/road240.png': 'unrecognised', './dataset/road245.png': 'red_circle', './d

### Finding a perfect circle

By using the HoughCircles function we are able to detect perfect circles in an image.

There are some cases where it fits very well but if the sign is slanted no decent circle can be found

In [68]:
circles =  cv.HoughCircles(gray_blurred, cv.HOUGH_GRADIENT, 1.5, 100)
output = img.copy()
# ensure at least some circles were found
print(circles)
if circles is not None:
    # convert the (x, y) coordinates and radius of the circles to integers
    circles = np.round(circles[0, :]).astype("int")
    # loop over the (x, y) coordinates and radius of the circles
    for (x, y, r) in circles:
        # draw the circle in the output image, then draw a rectangle
        # corresponding to the center of the circle
        cv.circle(output, (x, y), r, (0, 255, 0), 4)
        cv.rectangle(output, (x - 5, y - 5), (x + 5, y + 5), (0, 128, 255), -1) #center of circle
        # show the output image
    # cv.imshow("output", np.hstack([img, output]))

# cv.waitKey(0)
# cv.destroyAllWindows()

NameError: name 'gray_blurred' is not defined

### Commented out cell

To start we need to collect all of the xml files and the corresponding png files so we can parse them.
Our dataset has 877 pictures of traffic sings with 4 distinctions: 'trafficlight', 'speedlimit', 'crosswalk' and 'stop'.

In [ ]:
# xml_files = [name for name in os.listdir('./Dataset/annotations')]
# validation_dictionary = {}

# for name in xml_files:
#     # Parse the xml
#     mytree = ET.parse('./Dataset/annotations/' + name)
    
#     # Get number from name
#     order = ''.join(i for i in name if i.isdigit())
    
#     # Get sign type
#     validation_dictionary[order] = mytree.getroot().find('object').find('name').text

# sign_types = []
# for entry in validation_dictionary:
#     if validation_dictionary[entry] not in sign_types:
#         sign_types.append(validation_dictionary[entry])
# print(sign_types)
# print(validation_dictionary)